In [1]:
# COSC202 Project Fall 2024
# Testing NoteBook
# Group 5:
# - Abd Alrahman Ismaik     (1000#####)
# - Abdulrahman Abdulrahman (1000#####)
# - Abdulaziz Alyazidi      (1000#####)

In [2]:
import pandas as pd
file = "Datasets\\adult_real.csv"
data = pd.read_csv(file)

In [3]:
import numpy as np
data.replace(' ?', np.nan, inplace=True)

In [4]:
data = data.dropna(how='all')

In [5]:
data['capital-loss'] = np.log1p(data['capital-loss'])

In [6]:
data['age'] = data['age'].abs()

In [7]:
binary_mapping = {' <=50K.': 0.0, ' >50K.': 1.0}
data['class'] = data['class'].map(binary_mapping)

In [8]:
data.drop_duplicates(inplace= True)

In [9]:
education_mapping = {
    ' Preschool': 0,
    ' 1st-4th': 1,
    ' 5th-6th': 2,
    ' 7th-8th': 3,
    ' 9th': 4,
    ' 10th': 5,
    ' 11th': 6,
    ' 12th': 7,
    ' HS-grad': 8,
    ' Some-college': 9,
    ' Assoc-voc': 10,
    ' Assoc-acdm': 11,
    ' Bachelors': 12,
    ' Masters': 13,
    ' Prof-school': 14,
    ' Doctorate': 15
}

data['education'] = data['education'].map(education_mapping)

In [10]:
data = data.drop("education-num", axis=1)
data = data.drop("fnlwgt", axis=1)
data = data.drop("capital-gain", axis=1)

In [11]:
data['occupation'] = data['occupation'].fillna(' Missing')

In [12]:
data['sex'] = data['sex'].apply(lambda x: x if x in [" Male", " Female"] else " Missing")

In [13]:
data["race"] = data['race'].fillna(" Other")

In [14]:
most_frequent_country = data['native-country'].mode()[0]
data['native-country'] = data['native-country'].fillna(most_frequent_country)
data['native-country'] = data['native-country'].apply(lambda x: 1.0 if x == ' United-States' else 0.0)

In [15]:
data['workclass'] = data['workclass'].fillna(" Private")

In [16]:
# load the imputer
import pickle

with open('median_imputer.pkl', 'rb') as f:
    loaded_imputer = pickle.load(f)

numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[numerical_cols] = loaded_imputer.transform(data[numerical_cols])

In [17]:
# One Hot encode the remaining categorical data
data_encoded = pd.get_dummies(
    data, 
    columns=['workclass', 'marital-status', 'occupation',
       'relationship', 'race', 'sex'],
    )

data_encoded = data_encoded.astype(float)

In [18]:
data_encoded = data_encoded.drop("workclass_ Without-pay", axis=1)
data_encoded = data_encoded.drop("sex_ Missing", axis=1)
data_encoded = data_encoded.drop("marital-status_ Married-AF-spouse", axis=1)

In [19]:
X = data_encoded.drop("class",axis=1)
y = data_encoded["class"]

In [20]:
with open("scaler.pkl","rb") as file:
    loaded_imputer = pickle.load(file)

X = loaded_imputer.transform(X)
X

array([[ 0.46310356,  0.14922965, -0.26318227, ...,  0.44945799,
        -0.59307542,  0.67553839],
       [ 0.08194977,  0.53726014, -0.26318227, ...,  0.44945799,
        -0.59307542,  0.67553839],
       [-0.83281933, -0.23880084, -0.26318227, ...,  0.44945799,
        -0.59307542,  0.67553839],
       ...,
       [-0.1467425 ,  0.92529063, -0.26318227, ...,  0.44945799,
         1.75075602, -1.55452902],
       [-1.67135767, -2.17895329, -0.26318227, ...,  0.44945799,
        -0.59307542,  0.67553839],
       [ 0.99671887, -2.56698378, -0.26318227, ...,  0.44945799,
         1.75075602, -1.55452902]])

In [21]:
from tensorflow import keras
loaded_model = keras.models.load_model("trained_model.keras")
loaded_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 256)            │        11,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 430,853 (1.64 MB)

 Trainable params: 143,617 (561.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 287,236 (1.10 MB)

In [22]:
predictions = loaded_model.predict(X)

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [23]:
test_loss, test_acc = loaded_model.evaluate(X, y)
print(f'Test accuracy: {test_acc:.4f}')

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.8016 - loss: 0.6434
Test accuracy: 0.8023
